# PUMAP

Load, edit and test PUMAP ONNX model


In [4]:
import onnxruntime as ort
import onnx
import anndata

In [ ]:
# Change the model from fixed to variabe/batched inputs and outputs
model = onnx.load(
    "../checkpoints/allen-celltypes+human-cortex+various-cortical-areas-pumap.onnx"
)

model.graph.input[0].type.tensor_type.shape.dim[0].ClearField("dim_value")
model.graph.output[0].type.tensor_type.shape.dim[0].ClearField("dim_value")

ONNX_PATH = "dynamic_model.onnx"
onnx.save(
    model,
    "../public/models/allen-celltypes+human-cortex+various-cortical-areas-pumap.onnx",
)